# Cell Line Exp-PTM Distance Comparisons
There are 37 cell lines that are measured both in both CST (post-translational-modification, PTM) and CCLE (gene expression) datasets. We expect that the cell lines should be arranged similarly in PTM-space and gene-expression-space - in other words cell line distances measured based on PTM or gene-expression data should be similar. 

Since the gene expression data is not missing any data and has already been pre-normalized by the CCLE, we trust gene expression data more than PTM data. Furthermore, we can use cell-line distances in gene-expression-space as a measure for how well we have processed the PTM data. For instance, we would expect that if a normalization procedure is improving the data 